In [1]:
from maelzel.core import *

In [2]:
getPreset('?')

Preset: .sing  (Simple vowel singing simulation)
    routing=True
  init: gi__formantFreqs__[] fillarray \
            668, 1191, 2428, 3321, 4600, \  ; A
            327, 2157, 2754, 3630, 4600, \  ; E 
            208, 2152, 3128, 3425, 4200, \  ; I
            335, 628, 2689, 3515, 4200, \   ; O
            254, 796, 2515, 3274, 4160      ; U
                       
        gi__formantDbs__[] fillarray   \
            28, 28, 22, 20, 20, \
            15, 25, 24, 20, 23, \
            10, 20, 27, 26, 20, \
            15, 18, 5,  7,  12, \
            12, 10, 6,  5,  12
                                   
        gi__formantBws__[] fillarray   \
            80, 90, 120, 130, 140, \
            60, 100, 120, 150, 200, \
            60, 90, 100, 120, 120, \
            40, 80, 100, 120, 120, \
            50, 60, 170, 180, 200
            
        gi__formantAmps__[] maparray gi__formantDbs__, "ampdb"
        reshapearray gi__formantFreqs__, 5, 5
        reshapearray gi__formantAmps__, 5, 5
        reshapearray gi__formantBws__, 5, 5
  |kx=0.0, ky=0.0, kvibamount=1.0, ivibstart=0.5, ivibfreq=4.5, ivibrange=0.25, ipitchlag=0.2|
  ; Simple vowel singing simulation
  ; Args:
  ;   kx: x coordinate, from 0 to 1, A=(0;0), E=(0.5;0.5), I=(1;0), O=(0;1), U=(1;1)
  ;   ky: y coordinate, from 0 to 1, A=(0;0), E=(0.5;0.5), I=(1;0), O=(0;1), U=(1;1)
  ;   kvibamount: vibrato amount, 0 to 1
  ;   ivibstart: start time of vibrato
  ;   ivibfreq: vibrato frequency
  ;   ivibrange: vibrato range in semitones
  ;   ipitchlag: time lag for pitch modifications

  knoVib = lag:k(trighold(changed2(kpitch), ivibstart*0.8), ivibstart*0.2)

  kvibfreq = linseg:k(0, ivibstart*0.25, 0, ivibstart*0.75, ivibfreq) * randomi:k(0.9, 1.1, 2) * (1 - knoVib)
  kvibsemi = linseg:k(0, ivibstart*0.2, 0, ivibstart*0.8, ivibrange) * randomi:k(0.9, 1.1, 10)
  kvib = oscil:k(kvibsemi/2, kvibfreq) - kvibsemi/2
  kpitch2 = lag:k(kpitch, ipitchlag) + kvib*kvibamount
  asource = butterlp:a(vco2:a(kamp, mtof(kpitch2)), 5000)
  kcoords[] fillarray 0, 0, 1,       \  ; A
                      0.5, 0.5, 0.3, \  ; E
                      1, 0, 1,       \  ; I
                      0, 1, 1,       \  ; O
                      1, 1, 1           ; U

  kweights[] presetinterp kx, ky, kcoords, 0.2
  kformantFreqs[] weightedsum gi__formantFreqs__, kweights
  kformantBws[]   weightedsum gi__formantBws__, kweights
  kformantAmps[]  weightedsum gi__formantAmps__, kweights
  kformantFreqs poly 5, "lag", kformantFreqs, ipitchlag
  kformantAmps  poly 5, "lag", kformantAmps, ipitchlag
  aformants[] poly 5, "resonx", asource, kformantFreqs, kformantBws, 2, 1
  aformants *= kformantAmps
  aout1 = sumarray(aformants) * 0.1

In [2]:
presetManager.showPresets('*', full=True)


In [3]:
presetManager

In [3]:
s = playSession(latency=0)




--Csound version 6.19 (double samples) Oct 30 2023
[commit: 79e5d31bea441e5289073319ab2b1e42d01814b9]
libsndfile-1.2.0
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 442.0
audio buffered in 512 sample-frame blocks
reading 4096-byte blks of shorts from adc:Built-in Audio Pro: (RAW)
writing 1024 sample blks of 64-bit floats to dac:Built-in Audio Pro:
SECTION 1:


In [4]:
foo = defPreset('foo', r'''
|ibusL, ibusR|
prints "ibusL: %d, ibusR: %d\n", ibusL, ibusR
aout1 vco2 kamp, kfreq
aout2 vco2 kamp, kfreq+4

busout ibusL, aout1
busout ibusR, aout2
''', output=False)

In [5]:
busL = s.assignBus()
busR = s.assignBus()

s.defInstr('master', r'''
    |ibusL, ibusR|
    aL busin ibusL
    aR busin ibusR
    outch 1, aL, 2, aR
    ''')

Instr(master, ibusL:5=0, ibusR:6=0)

In [6]:
master = s.sched('master', args=dict(ibusL=busL, ibusR=busR), priority=2)

In [7]:
Note(60.5, 10).play(instr='foo', gain=0.01, ibusL=busL, ibusR=busR)

ibusL: 0, ibusR: 1


Button(description='Stop', style=ButtonStyle())

Output()

SynthGroup(n=1)
    Synth(▶ preset:foo=501.0002 start=5.229 dur=10.000 p4=0 kpos=0.5 kgain=0.01 idataidx_=17 inumbps=2 ibplen=3 ichan=1 …)

In [8]:
master.stop()

In [9]:
foo = s.defInstr('foo', r'''
    |itime, Sfoo|
    ielapsed times
    prints "Sfoo: %s, itime: %f, ielapsed: %f, diff: %f\n", Sfoo, itime, ielapsed, ielapsed - itime 
    turnoff
''', priority=1)

In [12]:
s.sched("foo", 0, 1, args={'Sfoo': 'bar', 'itime': s.engine.realElapsedTime()})

Button(description='Stop', style=ButtonStyle())

Output()

Synth(▶ foo=502.0004 start=16.556 dur=1.000 p4=0 itime=16.556 Sfoo=bar)

Sfoo: bar, itime: 16.556000, ielapsed: 16.556000, diff: 0.000000


In [33]:
s.engine.extraLatency

0

In [34]:
s.engine._strToIndex

{'cos': 20, 'linear': 21, 'smooth': 22, 'smoother': 23, 'bar': 24}

In [13]:
test = s.defInstr('test', r'''
    |istrid|
    S0 = strget(istrid)
    prints "S0: %s\n", S0
    turnoff
''')

In [14]:
jiji = s.engine.strSet('jiji')

In [16]:
s.sched('test', 0, 1, args={'istrid': jiji})

S0: jiji


Button(description='Stop', style=ButtonStyle())

Output()

Synth(▶ test=503.0003 start=26.753 dur=1.000 p4=0 istrid=25)

In [18]:
test = s.defInstr('test', r'''
    |ichan=0|
    prints "ichan: %d\n", ichan
    Schan = "bar"
    ; Schan strget ichan
    prints "Schan: %s\n", Schan
    a0 oscili 0.1, 1000
    ; chnset a0, Schan
''')


In [19]:
print(s.generateInstrBody(test))

ichan = p5
prints "ichan: %d\n", ichan
Schan = "bar"
; Schan strget ichan
prints "Schan: %s\n", Schan
a0 oscili 0.1, 1000
; chnset a0, Schan
atstop 1, 0.01, 0, p1


In [20]:
ichan = s.engine.strSet('foo')
ichan

26

In [22]:
s.sched('test', args={'ichan': 0})

ichan: 0
Schan: bar


Button(description='Stop', style=ButtonStyle())

Output()

Synth(▶ test=503.0004 start=96.705 dur=-1.000 p4=0 ichan=0)

In [24]:
s.testAudio()

Button(description='Stop', style=ButtonStyle())

Output()

Synth(▶ .testAudio=504.0002 start=104.087 dur=20.000 p4=0 imode=0 iperiod=1 igain=0.1)


Testaudio: pink noise mode
Channel: 1 / 2
Channel: 2 / 2
Channel: 1 / 2
Channel: 2 / 2
Channel: 1 / 2
Channel: 2 / 2
Channel: 1 / 2
Channel: 2 / 2
Channel: 1 / 2
Channel: 2 / 2
Channel: 1 / 2
Channel: 2 / 2
Channel: 1 / 2
Channel: 2 / 2
Channel: 1 / 2
Channel: 2 / 2
Channel: 1 / 2


In [27]:
bus = s.assignBus()
bus

Bus('audio', token=3)

In [30]:
Note(60, 1000).play(instr='foo', fade=0.05, args={'ibus': bus})

KeyError: "pfield 'ibus' not known for instr 'preset:foo'.Defined named pargs: dict_keys(['kpos', 'kgain', 'idataidx_', 'inumbps', 'ibplen', 'ichan', 'ifadein', 'ifadeout', 'ipchintrp_', 'ifadekind', 'ibusL', 'ibusR']) Aliases: {'position': 'kpos', 'gain': 'kgain'}"

In [31]:
s.defInstr('master', r'''
    |ibus|
    a0 = busin(ibus)
    outch 1, a0
''')

Instr(master, ibus:5=0)

In [32]:
s.sched('master', priority=2)

Bus not found (token 0, kind=0). Assigned bus 2


Button(description='Stop', style=ButtonStyle())

Output()

Synth(▶ master=1001.0003 start=245.505 dur=-1.000 p4=0 ibus=0)

Exiting python, closing all active engines
... stopping maelzel.core


end of score.		   overall amps:  0.07711  0.07887
	   overall samples out of range:        0        0
0 errors in performance
442323 1024 sample blks of 64-bit floats written to dac:Built-in Audio Pro:


In [12]:
post

Button(description='Stop', style=ButtonStyle())

Output()

Synth(▶ post:2001.0002, p4=0 ibus:5=0)

In [15]:
Note(72, dur=10).play(instr='foo', params=dict(ibus=busnum))


Button(description='Stop', style=ButtonStyle())

Output()

SynthGroup(n=1)
    Synth(𝍪 preset.foo:1001.0002, p4=0 idataidx_:5=16 inumbps:6=2 ibplen:7=3 igain_:8=1 ichan_:9=1 ipos:10=-1 ifade0:11=0.02 ifade1:12=0.02 ipchintrp_:13=0 ifadekind_:14=1 ibus:15=0 p16=0 ...)

Csound tidy up: Segmentation fault
